In [36]:
import pandas as pd
pd.set_option('future.no_silent_downcasting', True)

In [37]:
# FAO Data
fao_fbs = pd.read_csv(r"..\data\FAOSTAT\processed\fao_fbs_relevant.csv")
fao_fs = pd.read_csv(r"..\data\FAOSTAT\processed\fao_fs_relevant.csv")
fao_emissions = pd.read_csv(r"..\data\FAOSTAT\processed\fao_emissions_relevant.csv")
fao_cpi = pd.read_csv(r"..\data\FAOSTAT\processed\fao_cpi_relevant.csv")
fao_population = pd.read_csv(r"..\data\FAOSTAT\processed\fao_population_relevant.csv")
fao_prices = pd.read_csv(r"..\data\FAOSTAT\processed\fao_prices_relevant.csv")
fao_production_indices = pd.read_csv(r"..\data\FAOSTAT\processed\fao_production_indices_relevant.csv")

In [38]:
# Other Data
f_waste = pd.read_excel(r"../data/Food Waste Estimation UNEP 2024.xlsx")
fsi_data = pd.read_csv(r"../data/processed/fsi_data_melted.csv")
asean_sdg = pd.read_excel(r"../data/SDG From ASEANstats.xlsx")
un_sdg = pd.read_excel(r"../data/SDG From UNSDG (2, 11, 12, 13).xlsx")
gfsi_data = pd.read_csv(r"../data/processed/gfsi_data_melted.csv")
fsi_score = pd.read_csv(r"../data/processed/fsi_score_melted.csv")
gfsi_score = pd.read_csv(r"../data/processed/gfsi_score_melted.csv")

In [39]:
fao_fs['Year']

0         2000-2002
1         2001-2003
2         2002-2004
3         2003-2005
4         2004-2006
            ...    
283875    2016-2018
283876    2017-2019
283877    2018-2020
283878    2019-2021
283879    2020-2022
Name: Year, Length: 283880, dtype: object

In [40]:
# Replace all Lao People's Dem. Rep. with Lao People's Democratic Republic
f_waste['Country'] = f_waste['Country'].replace('Lao People\'s Dem. Rep.', 'Lao People\'s Democratic Republic')

# Exploratory Data Analysis (EDA)

## Urban Paradox

In [41]:
# Define the list of ASEAN countries for consistent filtering across all datasets.
asean_countries = [
    'Brunei Darussalam', 'Cambodia', 'Indonesia', 'Lao People\'s Democratic Republic',
    'Malaysia', 'Myanmar', 'Philippines', 'Singapore', 'Thailand', 'Viet Nam'
]

# Process Urban Population Data
max_actual_year = fao_population[fao_population['Year'] <= 2023]['Year'].max()
population_urban = fao_population[
    (fao_population['Area'].isin(asean_countries)) &
    (fao_population['Element'] == 'Urban population') &
    (fao_population['Year'] == 2024)
].copy()
population_urban['Value'] *= 1000  # Convert from '1000 persons' to persons
population_urban = population_urban[['Area', 'Year', 'Value']].rename(columns={
    'Area': 'Country',
    'Year': 'Population Year',
    'Value': 'Urban Population'
})

# Process Urban Food Insecurity Data
fs_urban_insecurity = fao_fs[
    (fao_fs['Area'].isin(asean_countries)) &
    (fao_fs['Item'] == 'Prevalence of moderate or severe food insecurity in the total population (percent) (3-year average)') &
    (fao_fs['Year'].astype(str).str.contains('2021|2022|2023|2024'))
].sort_values('Year').drop_duplicates('Area', keep='last')
fs_urban_insecurity = fs_urban_insecurity[['Area', 'Year', 'Value']].rename(columns={
    'Area': 'Country',
    'Year': 'FS Year',
    'Value': 'Urban Food Insecurity (%)'
})

# Process Food Waste Data
waste_household = f_waste[f_waste['Country'].isin(asean_countries)][['Country', 'Household estimate (kg/capita/year)']].rename(
    columns={'Household estimate (kg/capita/year)': 'Household Waste (kg/capita/yr)'}
)

# Merge Datasets to Create the Urban Paradox DataFrame
urban_paradox = population_urban.merge(fs_urban_insecurity, on='Country', how='left').merge(waste_household, on='Country', how='left')
urban_paradox['Total Household Waste (tonnes/yr)'] = (urban_paradox['Urban Population'] * urban_paradox['Household Waste (kg/capita/yr)']) / 1000
urban_paradox = urban_paradox.loc[:, ~urban_paradox.columns.str.contains('Year')]

urban_paradox_fmt = urban_paradox.copy()
urban_paradox_fmt['Urban Population'] = urban_paradox_fmt['Urban Population'].apply(lambda x: f"{x:,.0f}")
urban_paradox_fmt['Total Household Waste (tonnes/yr)'] = urban_paradox_fmt['Total Household Waste (tonnes/yr)'].apply(lambda x: f"{x:,.0f}")

print("--- Urban Paradox Data Ready for Visualization ---")
urban_paradox_fmt

--- Urban Paradox Data Ready for Visualization ---


,Country,Urban Population,Urban Food Insecurity (%),Household Waste (kg/capita/yr),Total Household Waste (tonnes/yr)
0,Brunei Darussalam,"368,738",NaN,76,"28,024"
1,Cambodia,"4,581,944",53.5,85,"389,465"
2,Indonesia,"167,176,839",5.4,53,"8,860,372"
3,Lao People's Democratic Republic,"2,933,677",39.1,89,"261,097"
4,Malaysia,"27,360,379",14.2,81,"2,216,191"
5,Myanmar,"18,372,626",36.9,78,"1,433,065"
6,Philippines,"56,434,179",42.7,26,"1,467,289"
7,Singapore,"6,119,203",7.7,68,"416,106"
8,Thailand,"37,838,857",6.7,86,"3,254,142"
9,Viet Nam,"40,975,863",12.3,72,"2,950,262"


## Estimated Financial Loss (Hybrid Price Filling) From Food Loss

In [42]:
# Filter the Food Balance Sheets for the 'Losses' element across all commodities
# in ASEAN countries. We will keep the 'Year' to be explicit about the timeframe.
loss_volume = fao_fbs[
    (fao_fbs['Area'].isin(asean_countries)) &
    (fao_fbs['Element'] == 'Losses') &
    (fao_fbs['Year'] == 2022) &
    (~fao_fbs['Item'].str.strip().str.lower().str.contains('total'))
].copy()
loss_volume['Value'] = loss_volume['Value'] * 1000
loss_volume = loss_volume.sort_values('Year').drop_duplicates(['Area', 'Item'], keep='last')
loss_volume = loss_volume[['Area', 'Item', 'Year', 'Value']].rename(columns={
    'Year': 'Loss Year',
    'Value': 'Losses (tonnes)'
})

# Filter the Producer Prices dataset for the USD/tonne value across all commodities
# in ASEAN, also keeping the 'Year'.
producer_prices = fao_prices[
    (fao_prices['Area'].isin(asean_countries)) &
    (fao_prices['Element'] == 'Producer Price (USD/tonne)')
].copy()
# For each (Area, Item), find the latest available price (any year)
producer_prices = (
    producer_prices
    .sort_values(['Area', 'Item', 'Year'], ascending=[True, True, False])
    .drop_duplicates(['Area', 'Item'], keep='first')
    .loc[:, ['Area', 'Item', 'Year', 'Value']]
    .rename(columns={'Year': 'Price Year', 'Value': 'Price (USD/tonne)'})
)

# Manual mapping from loss_volume items to producer_prices items (standardized, lowercased)
item_mapping = {
    'eggs': 'hen eggs in shell, fresh',
    'milk - excluding butter': 'raw milk of cattle',
    'fats, animals, raw': 'palm oil',
    'animal fats': 'palm oil',
    'rice and products': 'rice',
    'wheat and products': 'wheat',
    'cereals - excluding beer': 'cereals n.e.c.',
    'cassava and products': 'cassava, fresh',
    'starchy roots': 'edible roots and tubers with high starch or inulin content, n.e.c., fresh',
    'cereals, other': 'cereals n.e.c.',
    'sorghum and products': 'sorghum',
    'roots, other': 'edible roots and tubers with high starch or inulin content, n.e.c., fresh',
    'sweet potatoes': 'sweet potatoes',
    'potatoes and products': 'potatoes',
    'sugar crops': 'other sugar crops n.e.c.',
    'sugar cane': 'sugar cane',
    'yams': 'yams',
    'pulses': 'other pulses n.e.c.',
    'sweeteners, other': 'natural honey',
    'honey': 'natural honey',
    'sugar & sweeteners': 'sugar cane',
    'peas': 'peas, dry',
    'beans': 'beans, dry',
    'pulses, other and products': 'other pulses n.e.c.',
    'oilcrops': 'oil palm fruit',
    'treenuts': 'other nuts (excluding wild edible nuts and groundnuts), in shell, n.e.c.',
    'nuts and products': 'other nuts (excluding wild edible nuts and groundnuts), in shell, n.e.c.',
    'groundnuts': 'groundnuts, excluding shelled',
    'soyabeans': 'soya beans',
    'sunflower seed': 'sunflower seed',
    'coconuts - incl copra': 'coconuts, in shell',
    'cottonseed': 'cotton seed',
    'rape and mustardseed': 'rape or colza seed',
    'sesame seed': 'sesame seed',
    'oilcrops, other': 'other oil seeds, n.e.c.',
    'palm kernels': 'palm kernels',
    'vegetables': 'other vegetables, fresh n.e.c.',
    'palmkernel oil': 'palm kernels',
    'vegetable oils': 'oil palm fruit',
    'bananas': 'bananas',
    'onions': 'onions and shallots, dry (excluding dehydrated)',
    'fruits - excluding wine': 'other fruits, n.e.c.',
    'tomatoes and products': 'tomatoes',
    'grapefruit and products': 'pomelos and grapefruits',
    'vegetables, other': 'other vegetables, fresh n.e.c.',
    'lemons, limes and products': 'lemons and limes',
    'oranges, mandarines': 'oranges',
    'palm oil': 'palm oil',
    'pineapples and products': 'pineapples',
    'stimulants': 'other stimulant, spice and aromatic crops, n.e.c.',
    'plantains': 'plantains',
    'apples and products': 'apples',
    'fruits, other': 'other fruits, n.e.c.',
    'citrus, other': 'other citrus fruit, n.e.c.',
    'grapes and products (excl wine)': 'grapes',
    'coffee and products': 'coffee, green',
    'dates': 'dates',
    'spices': 'other stimulant, spice and aromatic crops, n.e.c.',
    'tea (including mate)': 'tea leaves',
    'spices, other': 'other stimulant, spice and aromatic crops, n.e.c.',
    'pimento': 'chillies and peppers, green (capsicum spp. and pimenta spp.)',
    'cocoa beans and products': 'cocoa beans',
    'pepper': 'pepper (piper spp.), raw',
    'cloves': 'cloves (whole stems), raw',
    'offals, edible': 'other meat of mammals, fresh or chilled',
    'offals': 'other meat of mammals, fresh or chilled',
    'meat': 'meat of chickens, fresh or chilled',
    'poultry meat': 'meat of chickens, fresh or chilled',
    'pigmeat': 'meat of pig with the bone, fresh or chilled',
    'millet and products': 'millet',
    'maize and products': 'maize (corn)',
    'barley and products': 'barley'
}

# Standardize and map items in loss_volume
loss_volume['Item_std'] = loss_volume['Item'].str.strip().str.lower()
loss_volume['Item_mapped'] = loss_volume['Item_std'].map(item_mapping)

# Only keep rows with a mapped item
loss_volume_mapped = loss_volume[loss_volume['Item_mapped'].notnull()].copy()

# Standardize producer_prices item names
producer_prices['Item_std'] = producer_prices['Item'].str.strip().str.lower()

# Hybrid approach: Fill missing prices with ASEAN and global averages for each item
# Calculate ASEAN average price for each item (excluding the country in question)
asean_avg_prices = (
    producer_prices.groupby('Item_std')['Price (USD/tonne)']
    .mean()
    .rename('ASEAN_Avg_Price')
    .reset_index()
)

# Calculate global average price for each item (from all available data)
global_prices = fao_prices[
    (fao_prices['Element'] == 'Producer Price (USD/tonne)')
].copy()
global_prices['Item_std'] = global_prices['Item'].str.strip().str.lower()
global_avg_prices = (
    global_prices.groupby('Item_std')['Value']
    .mean()
    .rename('Global_Avg_Price')
    .reset_index()
)

# Merge ASEAN and global averages into loss_volume_mapped
loss_volume_mapped = loss_volume_mapped.merge(
    asean_avg_prices, left_on='Item_mapped', right_on='Item_std', how='left', suffixes=('', '_ASEAN')
)
loss_volume_mapped = loss_volume_mapped.merge(
    global_avg_prices, left_on='Item_mapped', right_on='Item_std', how='left', suffixes=('', '_GLOBAL')
)

# Merge with producer_prices to get country-specific price if available
merged = pd.merge(
    loss_volume_mapped, producer_prices,
    left_on=['Area', 'Item_mapped'],
    right_on=['Area', 'Item_std'],
    how='left',
    suffixes=('', '_country')
)

# Choose price: country > ASEAN > global
def choose_price(row):
    if pd.notnull(row['Price (USD/tonne)']):
        return row['Price (USD/tonne)']
    elif pd.notnull(row['ASEAN_Avg_Price']):
        return row['ASEAN_Avg_Price']
    else:
        return row['Global_Avg_Price']

merged['Final_Price'] = merged.apply(choose_price, axis=1)
merged = merged[merged['Final_Price'].notnull()].copy()

merged['Financial Loss (USD)'] = merged['Losses (tonnes)'] * merged['Final_Price']

financial_loss_df_detailed = merged.rename(columns={'Area': 'Country'})
financial_loss_df_detailed = financial_loss_df_detailed.sort_values('Financial Loss (USD)', ascending=False)

print("--- Comprehensive Financial Loss Data (Hybrid Price Filling) ---")
financial_loss_df_detailed[['Country', 'Item', 'Loss Year', 'Final_Price', 'Losses (tonnes)', 'Financial Loss (USD)']]

--- Comprehensive Financial Loss Data (Hybrid Price Filling) ---


,Country,Item,Loss Year,Final_Price,Losses (tonnes),Financial Loss (USD)
172,Indonesia,Palm kernels,2022,281.400000,35611000.0,1.002094e+10
140,Indonesia,Oilcrops,2022,153.700000,37250000.0,5.725325e+09
17,Viet Nam,Cereals - Excluding Beer,2022,1170.472028,4412000.0,5.164123e+09
23,Indonesia,Cereals - Excluding Beer,2022,1170.472028,3988000.0,4.667842e+09
26,Thailand,Starchy Roots,2022,891.478049,2604000.0,2.321409e+09
...,...,...,...,...,...,...
102,Myanmar,Honey,2022,7135.400000,0.0,0.000000e+00
110,Cambodia,Treenuts,2022,7483.200000,0.0,0.000000e+00
120,Malaysia,Groundnuts,2022,1247.016667,0.0,0.000000e+00
115,Philippines,Peas,2022,531.260575,0.0,0.000000e+00


In [43]:
# Summarize Financial Losses with formatted output
financial_loss_by_country = financial_loss_df_detailed.groupby('Country')['Financial Loss (USD)'].sum().reset_index()
financial_loss_by_country = financial_loss_by_country.sort_values('Financial Loss (USD)', ascending=False)

# Format the numbers with commas and no scientific notation
financial_loss_by_country_fmt = financial_loss_by_country.copy()
financial_loss_by_country_fmt['Financial Loss (USD)'] = financial_loss_by_country_fmt['Financial Loss (USD)'].apply(lambda x: f"{x:,.0f}")

print("--- Total Financial Loss by Country (USD) ---")
financial_loss_by_country_fmt

--- Total Financial Loss by Country (USD) ---


,Country,Financial Loss (USD)
1,Indonesia,"28,406,356,192"
7,Viet Nam,"13,017,506,382"
6,Thailand,"7,635,115,728"
4,Myanmar,"5,893,983,708"
5,Philippines,"3,290,868,129"
0,Cambodia,"2,121,899,391"
3,Malaysia,"1,762,320,491"
2,Lao People's Democratic Republic,"1,651,359,452"


### Archived
> **Note:**  
The code below was previously used to debug unwanted output during financial loss calculation. It is no longer needed, as the desired output has been achieved.

In [44]:
print(loss_volume.shape)
print(financial_loss_df_detailed.shape)

(387, 6)
(387, 16)


In [45]:
# Diagnostic: Check for loss_volume rows that are dropped after mapping and merging
print(f"Rows in loss_volume: {len(loss_volume)}")
print(f"Rows in loss_volume_mapped: {len(loss_volume_mapped)}")
print(f"Rows in financial_loss_df_detailed: {len(financial_loss_df_detailed)}")

Rows in loss_volume: 387
Rows in loss_volume_mapped: 387
Rows in financial_loss_df_detailed: 387


In [46]:
# Diagnostic: Show which standardized item names from loss_volume are not present in the mapping dictionary
unmapped_items = set(loss_volume['Item_std'].unique()) - set(item_mapping.keys())
print("Items in loss_volume that are NOT present in item_mapping:")
print(sorted(unmapped_items))
print(f"Count: {len(unmapped_items)}")

# Diagnostic: Show which mapped values are not present in the price data (global_avg_prices['Item_std'])
mapped_values = set(item_mapping.values())
price_items = set(global_avg_prices['Item_std'].unique())
not_in_price = mapped_values - price_items
print("\nMapped values NOT present in price data (global_avg_prices['Item_std']):")
print(sorted(not_in_price))
print(f"Count: {len(not_in_price)}")

Items in loss_volume that are NOT present in item_mapping:
[]
Count: 0

Mapped values NOT present in price data (global_avg_prices['Item_std']):
[]
Count: 0


In [47]:
# For each item in not_in_price, check if it exists in fao_prices['Item'] with any other element (not 'Producer Price (USD/tonne)')
not_in_price_items = list(not_in_price)
other_element_rows = []

for item in not_in_price_items:
    # Find all rows in fao_prices where Item_std matches and Element is NOT 'Producer Price (USD/tonne)'
    mask = (
        (fao_prices['Item'].str.strip().str.lower() == item)
        & (fao_prices['Element'] != 'Producer Price (USD/tonne)')
    )
    matches = fao_prices[mask]
    if not matches.empty:
        other_element_rows.append((item, matches['Element'].unique().tolist()))

# Print the results
for item, elements in other_element_rows:
    print(f"Item '{item}' found in fao_prices with elements (not Producer Price): {elements}")

if not other_element_rows:
    print("No not_in_price items found in fao_prices with other elements.")

No not_in_price items found in fao_prices with other elements.


## Estimated Social Impact of Redistributing Edible Food Waste

### Household Waste Only

In [48]:
# Assumptions for estimation

# ASEAN average from Indonesia, Malaysia, Thailand studies (edible fraction, kcal/kg)
EDIBLE_WASTE_PERCENTAGE = 0.38  # Edible portion of household waste
KCAL_PER_KG_WASTE = 1477        # Caloric value per kg edible waste

asean_countries = [
    'Brunei Darussalam', 'Cambodia', 'Indonesia', 'Lao People\'s Democratic Republic',
    'Malaysia', 'Myanmar', 'Philippines', 'Singapore', 'Thailand', 'Viet Nam'
]

# Load household waste data
household_waste = f_waste[f_waste['Country'].isin(asean_countries)][['Country', 'Household estimate (tonnes/year)']].copy()
household_waste['Household estimate (tonnes/year)'] = household_waste['Household estimate (tonnes/year)'].astype(float)

# Latest daily energy requirement per country
daily_req = fao_fs[
    (fao_fs['Area'].isin(asean_countries)) &
    (fao_fs['Item'] == 'Minimum dietary energy requirement  (kcal/cap/day)')
].sort_values('Year').drop_duplicates('Area', keep='last')
daily_req['Value'] = daily_req['Value'].astype(float)
daily_req = daily_req[['Area', 'Value']].rename(columns={'Value': 'Daily Caloric Requirement (kcal)'})

# Latest food insecure population per country
insecure_pop = fao_fs[
    (fao_fs['Area'].isin(asean_countries)) &
    (fao_fs['Item'] == 'Number of moderately or severely food insecure people (million) (3-year average)')
].sort_values('Year').drop_duplicates('Area', keep='last')
insecure_pop['Value'] = insecure_pop['Value'].astype(float) * 1_000_000
insecure_pop = insecure_pop[['Area', 'Value']].rename(columns={'Value': 'Food Insecure Population'})

# Calculate potential people fed from edible food waste
social_impact_df = pd.merge(household_waste, daily_req, left_on='Country', right_on='Area')
social_impact_df['Edible Waste (kg/yr)'] = social_impact_df['Household estimate (tonnes/year)'] * 1000 * EDIBLE_WASTE_PERCENTAGE
social_impact_df['Total Wasted Calories (kcal/yr)'] = social_impact_df['Edible Waste (kg/yr)'] * KCAL_PER_KG_WASTE
social_impact_df['Annual Caloric Requirement (kcal/yr)'] = social_impact_df['Daily Caloric Requirement (kcal)'] * 365
social_impact_df['People Fed Annually (from waste)'] = (social_impact_df['Total Wasted Calories (kcal/yr)'] / social_impact_df['Annual Caloric Requirement (kcal/yr)']).round(0)

# Merge with food insecure population for context
social_impact_df = pd.merge(social_impact_df, insecure_pop, left_on='Country', right_on='Area', how='left')
social_impact_df['Insecure Population Coverage (%)'] = (social_impact_df['People Fed Annually (from waste)'] / social_impact_df['Food Insecure Population']) * 100

# Format output
social_impact_household = social_impact_df[[
    'Country',
    'People Fed Annually (from waste)',
    'Food Insecure Population',
    'Insecure Population Coverage (%)'
]].sort_values('People Fed Annually (from waste)', ascending=False).reset_index(drop=True)

social_impact_household_fmt = social_impact_household.copy()
social_impact_household_fmt['People Fed Annually (from waste)'] = social_impact_household['People Fed Annually (from waste)'].map('{:,.0f}'.format)
social_impact_household_fmt['Food Insecure Population'] = social_impact_household['Food Insecure Population'].map('{:,.0f}'.format)
social_impact_household_fmt['Insecure Population Coverage (%)'] = social_impact_household['Insecure Population Coverage (%)'].map('{:.2f}%'.format)

print("--- Estimated Social Impact of Redistributing Edible Food Waste ---")
social_impact_household_fmt

--- Estimated Social Impact of Redistributing Edible Food Waste ---


,Country,People Fed Annually (from waste),Food Insecure Population,Insecure Population Coverage (%)
0,Indonesia,"12,610,125","15,000,000",84.07%
1,Viet Nam,"6,085,308","12,100,000",50.29%
2,Thailand,"5,033,738","4,800,000",104.87%
3,Myanmar,"3,600,710","20,000,000",18.00%
4,Philippines,"2,593,181","52,700,000",4.92%
5,Malaysia,"2,288,528","4,800,000",47.68%
6,Cambodia,"1,222,437","9,000,000",13.58%
7,Lao People's Democratic Republic,"567,131","2,900,000",19.56%
8,Singapore,"329,423","500,000",65.88%
9,Brunei Darussalam,"28,048",nan,nan%


### Total Food Waste

In [49]:
# Assumptions for estimation

# ASEAN-wide assumptions for edible waste composition and caloric density
EDIBLE_WASTE_PERCENTAGE = 0.5  # Edible fraction of consumer-level waste
KCAL_PER_KG_WASTE = 1477       # Caloric value per kg edible waste

asean_countries = [
    'Brunei Darussalam', 'Cambodia', 'Indonesia', 'Lao People\'s Democratic Republic',
    'Malaysia', 'Myanmar', 'Philippines', 'Singapore', 'Thailand', 'Viet Nam'
]

# Load total food waste data (household + retail + food service)
waste_cols = [
    'Household estimate (tonnes/year)',
    'Retail estimate (tonnes/year)',
    'Food service estimate (tonnes/year)'
]
total_waste = f_waste[f_waste['Country'].isin(asean_countries)][['Country'] + waste_cols].copy()
total_waste = total_waste.replace('-', pd.NA).infer_objects(copy=False)
for col in waste_cols:
    total_waste[col] = total_waste[col].astype('Int64')

# Impute missing Food service estimate (tonnes/year) using average ratio to (household + retail)
cols = [
    'Household estimate (tonnes/year)',
    'Retail estimate (tonnes/year)',
    'Food service estimate (tonnes/year)'
]
total_waste[cols] = total_waste[cols].apply(pd.to_numeric, errors='coerce')
mask = total_waste[cols].notna().all(axis=1)
avg_ratio = (
    total_waste.loc[mask, 'Food service estimate (tonnes/year)'] /
    (total_waste.loc[mask, 'Household estimate (tonnes/year)'] + total_waste.loc[mask, 'Retail estimate (tonnes/year)'])
).mean()
missing = total_waste['Food service estimate (tonnes/year)'].isna()
sum_hr = total_waste.loc[missing, 'Household estimate (tonnes/year)'] + total_waste.loc[missing, 'Retail estimate (tonnes/year)']
total_waste.loc[missing, 'Food service estimate (tonnes/year)'] = (sum_hr * avg_ratio).round().astype('Int64')

total_waste['Total Food Waste (tonnes/year)'] = total_waste[waste_cols].sum(axis=1)

# Latest daily energy requirement per country
daily_req = fao_fs[
    (fao_fs['Area'].isin(asean_countries)) &
    (fao_fs['Item'] == 'Minimum dietary energy requirement  (kcal/cap/day)')
].sort_values('Year').drop_duplicates('Area', keep='last')
daily_req['Value'] = daily_req['Value'].astype(float)
daily_req = daily_req[['Area', 'Value']].rename(columns={'Value': 'Daily Caloric Requirement (kcal)'})

# Latest food insecure population per country
insecure_pop = fao_fs[
    (fao_fs['Area'].isin(asean_countries)) &
    (fao_fs['Item'] == 'Number of moderately or severely food insecure people (million) (3-year average)')
].sort_values('Year').drop_duplicates('Area', keep='last')
insecure_pop['Value'] = insecure_pop['Value'].astype(float) * 1_000_000
insecure_pop = insecure_pop[['Area', 'Value']].rename(columns={'Value': 'Food Insecure Population'})

# Calculate potential people fed from edible food waste (using total waste)
social_impact_df = pd.merge(total_waste, daily_req, left_on='Country', right_on='Area')
social_impact_df['Edible Waste (kg/yr)'] = social_impact_df['Total Food Waste (tonnes/year)'] * 1000 * EDIBLE_WASTE_PERCENTAGE
social_impact_df['Total Wasted Calories (kcal/yr)'] = social_impact_df['Edible Waste (kg/yr)'] * KCAL_PER_KG_WASTE
social_impact_df['Annual Caloric Requirement (kcal/yr)'] = social_impact_df['Daily Caloric Requirement (kcal)'] * 365
social_impact_df['People Fed Annually (from waste)'] = (social_impact_df['Total Wasted Calories (kcal/yr)'] / social_impact_df['Annual Caloric Requirement (kcal/yr)']).round(0)

# Merge with food insecure population for context
social_impact_df = pd.merge(social_impact_df, insecure_pop, left_on='Country', right_on='Area', how='left')
social_impact_df['Insecure Population Coverage (%)'] = (social_impact_df['People Fed Annually (from waste)'] / social_impact_df['Food Insecure Population']) * 100

# Format output
social_impact_total = social_impact_df[[
    'Country',
    'People Fed Annually (from waste)',
    'Food Insecure Population',
    'Insecure Population Coverage (%)'
]].sort_values('People Fed Annually (from waste)', ascending=False).reset_index(drop=True)

social_impact_total_fmt = social_impact_total.copy()
social_impact_total_fmt['People Fed Annually (from waste)'] = social_impact_total['People Fed Annually (from waste)'].map('{:,.0f}'.format)
social_impact_total_fmt['Food Insecure Population'] = social_impact_total['Food Insecure Population'].map('{:,.0f}'.format)
social_impact_total_fmt['Insecure Population Coverage (%)'] = social_impact_total['Insecure Population Coverage (%)'].map('{:.2f}%'.format)

print("--- Estimated Social Impact of Redistributing Edible Food Waste ---")
social_impact_total_fmt

--- Estimated Social Impact of Redistributing Edible Food Waste ---


,Country,People Fed Annually (from waste),Food Insecure Population,Insecure Population Coverage (%)
0,Indonesia,"32,089,465","15,000,000",213.93%
1,Viet Nam,"12,921,298","12,100,000",106.79%
2,Thailand,"10,514,810","4,800,000",219.06%
3,Philippines,"9,344,638","52,700,000",17.73%
4,Myanmar,"8,718,702","20,000,000",43.59%
5,Malaysia,"7,171,796","4,800,000",149.41%
6,Cambodia,"2,467,351","9,000,000",27.42%
7,Lao People's Democratic Republic,"1,356,839","2,900,000",46.79%
8,Singapore,"741,118","500,000",148.22%
9,Brunei Darussalam,"145,046",nan,nan%


## Estimated Environmental Impact of Food Loss and Waste

### Method: Using GHG Emissions Factors of food loss/waste

In [55]:
# Emission factor: 2.5 tonnes CO2eq per tonne of food loss/waste
GHG_EMISSION_FACTOR = 2.5

asean_countries = [
    'Brunei Darussalam', 'Cambodia', 'Indonesia', 'Lao People\'s Democratic Republic',
    'Malaysia', 'Myanmar', 'Philippines', 'Singapore', 'Thailand', 'Viet Nam'
]

total_loss_per_country = loss_volume.groupby('Area')['Losses (tonnes)'].sum().reset_index()
total_loss_per_country = total_loss_per_country.rename(columns={'Area': 'Country', 'Losses (tonnes)': 'Total Food Loss (tonnes/year)'})

# Get total food waste per country from total_waste from previous analysis
total_waste_ghg = total_waste[['Country', 'Total Food Waste (tonnes/year)']].copy()

# Combine loss & waste, calculate GHG emissions
environmental_impact = pd.merge(total_loss_per_country, total_waste_ghg, on='Country', how='outer').fillna(0)
environmental_impact['Total Loss and Waste (tonnes)'] = (
    environmental_impact['Total Food Loss (tonnes/year)'] + environmental_impact['Total Food Waste (tonnes/year)']
)
environmental_impact['Estimated GHG Emissions (tonnes CO2eq)'] = (
    environmental_impact['Total Loss and Waste (tonnes)'] * GHG_EMISSION_FACTOR
)
environmental_impact = (
    environmental_impact
    .sort_values(by='Total Loss and Waste (tonnes)', ascending=False)
    .reset_index(drop=True)
)

# Format output
environmental_impact_fmt = environmental_impact.copy()
for col in [
    'Total Food Loss (tonnes/year)', 'Total Food Waste (tonnes/year)',
    'Total Loss and Waste (tonnes)', 'Estimated GHG Emissions (tonnes CO2eq)'
    ]:
    environmental_impact_fmt[col] = environmental_impact_fmt[col].map('{:,.0f}'.format)

print("--- Estimated Environmental Impact of Food Loss and Waste ---")
environmental_impact_fmt

--- Estimated Environmental Impact of Food Loss and Waste ---


,Country,Total Food Loss (tonnes/year),Total Food Waste (tonnes/year),Total Loss and Waste (tonnes),Estimated GHG Emissions (tonnes CO2eq)
0,Indonesia,"91,127,000","28,484,669","119,611,669","299,029,172"
1,Viet Nam,"17,835,000","11,425,069","29,260,069","73,150,172"
2,Thailand,"13,955,000","9,811,734","23,766,734","59,416,835"
3,Myanmar,"7,498,000","7,769,444","15,267,444","38,168,610"
4,Philippines,"7,013,000","8,091,685","15,104,685","37,761,712"
5,Malaysia,"4,151,000","6,561,101","10,712,101","26,780,252"
6,Cambodia,"3,159,000","2,177,984","5,336,984","13,342,460"
7,Lao People's Democratic Republic,"2,451,000","1,225,206","3,676,206","9,190,515"
8,Singapore,0,"699,621","699,621","1,749,052"
9,Brunei Darussalam,0,"134,057","134,057","335,142"


### Method: Using emissions per capita and population data (Waste Emissions)

In [349]:
asean_countries = [
    'Brunei Darussalam', 'Cambodia', 'Indonesia', 'Lao People\'s Democratic Republic',
    'Malaysia', 'Myanmar', 'Philippines', 'Singapore', 'Thailand', 'Viet Nam'
]

# Emissions per capita from waste (latest year per country)
emissions_per_capita = (
    fao_emissions[
        (fao_emissions['Area'].isin(asean_countries)) &
        (fao_emissions['Item'] == 'Waste') &
        (fao_emissions['Element'] == 'Emissions per capita')
    ]
    .sort_values('Year')
    .drop_duplicates('Area', keep='last')
    .rename(columns={'Area': 'Country', 'Year': 'Emission Data Year', 'Value': 'Emissions per Capita (tonnes CO2eq)'})
    [['Country', 'Emission Data Year', 'Emissions per Capita (tonnes CO2eq)']]
)

# Total population (2022)
total_population = (
    fao_population[
        (fao_population['Area'].isin(asean_countries)) &
        (fao_population['Element'] == 'Total Population - Both sexes')
    ]
)
latest_year = total_population[total_population['Year'].isin([2022])]['Year'].max()
total_population = (
    total_population[total_population['Year'] == latest_year]
    .sort_values('Year')
    .drop_duplicates('Area', keep='last')
    .rename(columns={'Area': 'Country', 'Year': 'Population Year', 'Value': 'Total Population'})
)
total_population['Total Population'] *= 1000
total_population = total_population[['Country', 'Population Year', 'Total Population']]

# Merge and calculate total emissions
ghg_via_per_capita = pd.merge(emissions_per_capita, total_population, on='Country')
ghg_via_per_capita['Total Emissions (tonnes CO2eq)'] = (
    ghg_via_per_capita['Emissions per Capita (tonnes CO2eq)'] * ghg_via_per_capita['Total Population']
)
ghg_via_per_capita = (
    ghg_via_per_capita
    .sort_values('Total Emissions (tonnes CO2eq)', ascending=False)
    .reset_index(drop=True)
)

ghg_via_per_capita_fmt = ghg_via_per_capita.copy()
ghg_via_per_capita_fmt['Total Emissions (tonnes CO2eq)'] = ghg_via_per_capita_fmt['Total Emissions (tonnes CO2eq)'].map('{:,.0f}'.format)
ghg_via_per_capita_fmt['Total Population'] = ghg_via_per_capita_fmt['Total Population'].map('{:,.0f}'.format)

print("--- Estimated GHG Emissions from Waste (Per Capita Method, tonnes) ---")
ghg_via_per_capita_fmt

--- Estimated GHG Emissions from Waste (Per Capita Method, tonnes) ---


,Country,Emission Data Year,Emissions per Capita (tonnes CO2eq),Population Year,Total Population,Total Emissions (tonnes CO2eq)
0,Indonesia,2022,0.33,2022,"278,830,529","92,014,075"
1,Thailand,2022,0.44,2022,"71,735,329","31,563,545"
2,Viet Nam,2022,0.28,2022,"99,680,655","27,910,583"
3,Philippines,2022,0.21,2022,"113,964,339","23,932,511"
4,Malaysia,2022,0.50,2022,"34,695,493","17,347,746"
5,Myanmar,2022,0.20,2022,"53,756,787","10,751,357"
6,Singapore,2022,0.85,2022,"5,649,885","4,802,402"
7,Cambodia,2022,0.20,2022,"17,201,724","3,440,345"
8,Lao People's Democratic Republic,2022,0.22,2022,"7,559,007","1,662,982"
9,Brunei Darussalam,2022,0.35,2022,"455,370","159,380"


### Method: Using emissions per capita and population data (Agrifood Systems Emissions)

In [350]:
asean_countries = [
    'Brunei Darussalam', 'Cambodia', 'Indonesia', 'Lao People\'s Democratic Republic',
    'Malaysia', 'Myanmar', 'Philippines', 'Singapore', 'Thailand', 'Viet Nam'
]

# Emissions per capita from waste (latest year per country)
emissions_per_capita = (
    fao_emissions[
        (fao_emissions['Area'].isin(asean_countries)) &
        (fao_emissions['Item'] == 'Agrifood systems') &
        (fao_emissions['Element'] == 'Emissions per capita')
    ]
    .sort_values('Year')
    .drop_duplicates('Area', keep='last')
    .rename(columns={'Area': 'Country', 'Year': 'Emission Data Year', 'Value': 'Emissions per Capita (tonnes CO2eq)'})
    [['Country', 'Emission Data Year', 'Emissions per Capita (tonnes CO2eq)']]
)

# Total population (2022)
total_population = (
    fao_population[
        (fao_population['Area'].isin(asean_countries)) &
        (fao_population['Element'] == 'Total Population - Both sexes')
    ]
)
latest_year = total_population[total_population['Year'].isin([2022])]['Year'].max()
total_population = (
    total_population[total_population['Year'] == latest_year]
    .sort_values('Year')
    .drop_duplicates('Area', keep='last')
    .rename(columns={'Area': 'Country', 'Year': 'Population Year', 'Value': 'Total Population'})
)
total_population['Total Population'] *= 1000
total_population = total_population[['Country', 'Population Year', 'Total Population']]

# Merge and calculate total emissions
agrifood_system_ghg = pd.merge(emissions_per_capita, total_population, on='Country')
agrifood_system_ghg['Total Emissions (tonnes CO2eq)'] = (
    agrifood_system_ghg['Emissions per Capita (tonnes CO2eq)'] * agrifood_system_ghg['Total Population']
)
agrifood_system_ghg = (
    agrifood_system_ghg
    .sort_values('Total Emissions (tonnes CO2eq)', ascending=False)
    .reset_index(drop=True)
)

agrifood_system_ghg_fmt = agrifood_system_ghg.copy()
agrifood_system_ghg_fmt['Total Emissions (tonnes CO2eq)'] = agrifood_system_ghg_fmt['Total Emissions (tonnes CO2eq)'].map('{:,.0f}'.format)
agrifood_system_ghg_fmt['Total Population'] = agrifood_system_ghg_fmt['Total Population'].map('{:,.0f}'.format)

print("--- Estimated GHG Emissions from Waste (Per Capita Method, tonnes) ---")
agrifood_system_ghg_fmt

--- Estimated GHG Emissions from Waste (Per Capita Method, tonnes) ---


,Country,Emission Data Year,Emissions per Capita (tonnes CO2eq),Population Year,Total Population,Total Emissions (tonnes CO2eq)
0,Indonesia,2022,2.86,2022,"278,830,529","797,455,313"
1,Thailand,2022,2.59,2022,"71,735,329","185,794,502"
2,Myanmar,2022,3.29,2022,"53,756,787","176,859,829"
3,Viet Nam,2022,1.52,2022,"99,680,655","151,514,596"
4,Malaysia,2022,3.55,2022,"34,695,493","123,169,000"
5,Philippines,2022,0.93,2022,"113,964,339","105,986,835"
6,Cambodia,2022,3.63,2022,"17,201,724","62,442,258"
7,Lao People's Democratic Republic,2022,3.74,2022,"7,559,007","28,270,686"
8,Singapore,2022,0.82,2022,"5,649,885","4,632,906"
9,Brunei Darussalam,2022,3.66,2022,"455,370","1,666,654"


## Estimated Annual Electricity Generation Potential

In [ ]:
# Assumptions and conversion factors
INEDIBLE_WASTE_PERCENTAGE = 0.5
BIOGAS_YIELD_M3_PER_TONNE = 120
METHANE_PERCENTAGE_IN_BIOGAS = 0.60
KWH_PER_M3_METHANE = 10.0
GENERATOR_EFFICIENCY = 0.35

# Calculate total food loss and waste per country
total_waste_df = total_loss_per_country.merge(
    total_waste[['Country', 'Total Food Waste (tonnes/year)']].rename(
        columns={'Total Food Waste (tonnes/year)': 'Total Food Waste (tonnes)'}
    ),
    on='Country', how='outer'
).fillna(0)
total_waste_df['Total Loss and Waste (tonnes)'] = (
    total_waste_df['Total Food Loss (tonnes)'] + total_waste_df['Total Food Waste (tonnes)']
)

# Calculate bioenergy potential
bioenergy = environmental_impact[['Country', 'Total Loss and Waste (tonnes)']].copy()
bioenergy['Inedible Waste (tonnes)'] = bioenergy['Total Loss and Waste (tonnes)'] * INEDIBLE_WASTE_PERCENTAGE
bioenergy['Biogas Potential (m³)'] = bioenergy['Inedible Waste (tonnes)'] * BIOGAS_YIELD_M3_PER_TONNE
bioenergy['Methane Potential (m³)'] = bioenergy['Biogas Potential (m³)'] * METHANE_PERCENTAGE_IN_BIOGAS
bioenergy['Gross Energy Potential (kWh)'] = bioenergy['Methane Potential (m³)'] * KWH_PER_M3_METHANE
bioenergy['Net Electricity Generated (kWh)'] = bioenergy['Gross Energy Potential (kWh)'] * GENERATOR_EFFICIENCY

bioenergy = bioenergy.sort_values('Net Electricity Generated (kWh)', ascending=False).reset_index(drop=True)

# Format output
bioenergy_fmt = bioenergy.copy()
bioenergy_fmt['Total Loss and Waste (tonnes)'] = bioenergy['Total Loss and Waste (tonnes)'].map('{:,.0f}'.format)
bioenergy_fmt['Net Electricity Generated (kWh)'] = bioenergy['Net Electricity Generated (kWh)'].map('{:,.0f}'.format)
bioenergy_fmt['Inedible Waste (tonnes)'] = bioenergy['Inedible Waste (tonnes)'].map('{:,.0f}'.format)
bioenergy_fmt['Biogas Potential (m³)'] = bioenergy['Biogas Potential (m³)'].map('{:,.0f}'.format)
bioenergy_fmt['Methane Potential (m³)'] = bioenergy['Methane Potential (m³)'].map('{:,.0f}'.format)
bioenergy_fmt['Gross Energy Potential (kWh)'] = bioenergy['Gross Energy Potential (kWh)'].map('{:,.0f}'.format)

print("--- Estimated Annual Electricity Generation Potential from Inedible Food Waste ---")
bioenergy_fmt

--- Estimated Annual Electricity Generation Potential from Inedible Food Waste ---


,Country,Total Loss and Waste (tonnes),Inedible Waste (tonnes),Biogas Potential (m³),Methane Potential (m³),Gross Energy Potential (kWh),Net Electricity Generated (kWh)
0,Indonesia,"28,575,796","16,002,446","1,920,293,491","1,152,176,095","11,521,760,947","4,032,616,332"
1,Viet Nam,"11,442,904","6,408,026","768,963,149","461,377,889","4,613,778,893","1,614,822,612"
2,Thailand,"9,825,689","5,502,386","660,286,301","396,171,780","3,961,717,805","1,386,601,232"
3,Philippines,"8,098,698","4,535,271","544,232,506","326,539,503","3,265,395,034","1,142,888,262"
4,Myanmar,"7,776,942","4,355,088","522,610,502","313,566,301","3,135,663,014","1,097,482,055"
5,Malaysia,"6,565,252","3,676,541","441,184,934","264,710,961","2,647,109,606","926,488,362"
6,Cambodia,"2,181,143","1,221,440","146,572,810","87,943,686","879,436,858","307,802,900"
7,Lao People's Democratic Republic,"1,227,657","687,488","82,498,550","49,499,130","494,991,302","173,246,956"
8,Singapore,"699,621","391,788","47,014,531","28,208,719","282,087,187","98,730,516"
9,Brunei Darussalam,"134,057","75,072","9,008,630","5,405,178","54,051,782","18,918,124"


## Regulations (Questionable)

In [370]:
asean_countries = [
    'Brunei Darussalam', 'Cambodia', 'Indonesia', 'Lao People\'s Democratic Republic',
    'Malaysia', 'Myanmar', 'Philippines', 'Singapore', 'Thailand', 'Viet Nam'
]

# Household Waste per Capita
waste_per_capita = f_waste[f_waste['Country'].isin(asean_countries)][['Country', 'Household estimate (kg/capita/year)']].rename(
    columns={'Household estimate (kg/capita/year)': 'Household Waste (kg/capita/yr)'}
)

# Food Insecurity Prevalence
fs_insecurity = fao_fs[
    (fao_fs['Area'].isin(asean_countries)) &
    (fao_fs['Item'] == 'Prevalence of moderate or severe food insecurity in the total population (percent) (3-year average)')
].sort_values('Year').drop_duplicates('Area', keep='last')[['Area', 'Value']].rename(
    columns={'Area': 'Country', 'Value': 'Food Insecurity (%)'}
)

# GHG Emissions per Capita
emissions_pc = fao_emissions[
    (fao_emissions['Area'].isin(asean_countries)) &
    (fao_emissions['Item'] == 'Waste') &
    (fao_emissions['Element'] == 'Emissions per capita')
].sort_values('Year').drop_duplicates('Area', keep='last')[['Area', 'Value']].rename(
    columns={'Area': 'Country', 'Value': 'Waste GHG Emissions (tonnes CO2eq/capita)'}
)

# Assemble the Country Performance Scorecard
scorecard_df = waste_per_capita.merge(fs_insecurity, on='Country', how='left').merge(emissions_pc, on='Country', how='left')
scorecard_df = scorecard_df.sort_values('Food Insecurity (%)').reset_index(drop=True)

print("--- ASEAN Country Performance Scorecard ---")
scorecard_df

--- ASEAN Country Performance Scorecard ---


,Country,Household Waste (kg/capita/yr),Food Insecurity (%),Waste GHG Emissions (tonnes CO2eq/capita)
0,Viet Nam,72,12.3,0.28
1,Malaysia,81,14.2,0.50
2,Myanmar,78,36.9,0.20
3,Lao People's Democratic Republic,89,39.1,0.22
4,Philippines,26,45.6,0.21
5,Indonesia,53,5.4,0.33
6,Cambodia,85,53.5,0.20
7,Thailand,86,6.7,0.44
8,Singapore,68,8.4,0.85
9,Brunei Darussalam,76,NaN,0.35


In [372]:
# Load GFSI Data for Profiling
asean_gfsi_countries = ['Cambodia', 'Indonesia', 'Malaysia', 'Myanmar', 'Philippines', 'Singapore', 'Thailand']
gfsi_asean = gfsi_data[gfsi_data['Country'].isin(asean_gfsi_countries)]
gfsi_asean.loc[:, 'Value'] = gfsi_asean['Value'].astype(float)

# Display key GFSI indicators for top-performing countries
top_performers = scorecard_df.head(4)['Country'].tolist()
key_gfsi_indicators = [
    '1.5.1) Presence of food safety net programmes',
    '2.3.1) Crop storage facilities',
    '2.6.2) Road infrastructure',
    '2.9.1) Food security strategy',
    '2.5) Food loss'
]

profiling_df = gfsi_asean[
    (gfsi_asean['Country'].isin(top_performers)) &
    (gfsi_asean['Indicator'].isin(key_gfsi_indicators))
]

# Pivot table for comparison
profiling_pivot = profiling_df.pivot_table(index='Indicator', columns='Country', values='Value')

print("\n--- GFSI Profile of Countries with Lower Food Insecurity ---")
profiling_pivot


--- GFSI Profile of Countries with Lower Food Insecurity ---


Country,Malaysia,Myanmar
Indicator,,
1.5.1) Presence of food safety net programmes,1.0,1.0
2.3.1) Crop storage facilities,1.0,0.0
2.5) Food loss,1.66,6.34
2.6.2) Road infrastructure,2.0,0.0
2.9.1) Food security strategy,1.0,1.0
